In [ ]:
from icecream import ic
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub

In [ ]:
ELMO = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
import pandas as pd
a = pd.read_csv('swb-train.csv')
a.head()

,wav_filename,wav_filesize,transcript
0,/home/abhijeet/Desktop/dl_projects/DeepSpeech/...,194632,do you have any special menus that you like to...
1,/home/abhijeet/Desktop/dl_projects/DeepSpeech/...,65240,yes but we dream
2,/home/abhijeet/Desktop/dl_projects/DeepSpeech/...,76738,we dream though
3,/home/abhijeet/Desktop/dl_projects/DeepSpeech/...,55188,yes
4,/home/abhijeet/Desktop/dl_projects/DeepSpeech/...,39888,well


In [ ]:
tokens = a['transcript'].to_list()

new_len = [len(t) for t in tokens]
embeddings = ELMO(inputs={
    "text": tokens,
}, signature="default", as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
vocabulary = []
for i in range(len(tokens)):
  for j in range(len(tokens[i].split(" "))):
    vocabulary.append(tokens[i].split(" ")[j])
print(len(vocabulary))

840


In [ ]:
weights = []
init = tf.global_variables_initializer()
for i in range(len(tokens)):
  for j in range(len(tokens[i].split(" "))):
    with tf.Session() as sess:
      sess.run(init)
      weights.append(sess.run(embeddings[i][j]))
print(weights)

[array([ 0.43084213,  0.35530674, -0.51716274, ..., -0.679351  ,
        0.25733703,  0.59420997], dtype=float32), array([-0.2707683 , -0.20358156,  0.03061542, ..., -0.20794724,
       -0.1158641 ,  0.14105737], dtype=float32), array([ 0.40785924,  0.5565176 , -0.34256703, ...,  0.11382306,
        0.0907148 ,  0.45528197], dtype=float32), array([ 0.63437355,  0.16778018, -0.39460742, ..., -0.11989012,
       -0.32354832,  0.12096103], dtype=float32), array([ 0.14705552,  0.4590198 , -0.0126584 , ..., -0.07692005,
       -0.2862506 , -0.05564619], dtype=float32), array([ 0.24749778,  0.4227458 ,  0.10602897, ...,  0.48150223,
        0.00140664, -0.00977785], dtype=float32), array([-4.1122735e-04,  4.1774738e-01,  1.8436624e-01, ...,
       -6.5479589e-01,  4.0498757e-01,  3.0041504e-01], dtype=float32), array([-1.1636693 , -0.09100143, -0.16946746, ..., -0.22294237,
        0.18308605, -0.0222709 ], dtype=float32), array([-0.02800708,  0.10634784,  0.14073637, ...,  0.00813811,
     

In [ ]:
import math
def euclidean(a, b):
  return math.sqrt(sum((a[i] - b[i]) ** 2 for i in range(len(a))))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def cosine(a, b):
  return cosine_similarity([a], [b])[0][0]

In [ ]:
with open('embeddings.txt', 'w') as f:
   f.write('{} {}\n'.format(len(vocabulary), len(weights[1])))
   for i in range(len(vocabulary)):
    f.write('{}'.format(vocabulary[i]))
    for weight in weights[i]:
      f.write(' {}'.format(weight))
    f.write('\n')

In [ ]:
import random
words = []
euclidean_distances = []
cosine_distances = []

for _ in range(5):
  int_pair = (random.randint(0, len(vocabulary)), random.randint(0, len(vocabulary)))
  pair = (vocabulary[int_pair[0]], vocabulary[int_pair[1]])
  words.append(pair)
  euclidean_distances.append(euclidean(weights[int_pair[0]], weights[int_pair[1]]))
  cosine_distances.append(cosine(weights[int_pair[0]], weights[int_pair[1]]))